Gabor ML - Problem 10

Outline:

Functions:
-Generate linearly separably data
-Generate non-linearly separably data
-Perceptron algorithm

Uses:
-test algorithm on different dimensions of data and types of data

In [3]:
#Libraries

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


path_out_ = r'/home/clarice/Documents/VSCode/Term2_Gabor_ML/homework3/GaborML_problem10/outputs'
random.seed(123)

In [2]:
#Functions

#Generate linearly separable data
def generate_ls_data(a, d, n):
    #a is a number in range [0,1]
    #d is dimension of sample data
    #n is number of sample points

    #x is a list of n numpy arrays
    #y is a list of n corresponding {-1,1}

    #determine y values with probability .5
    probability_y_equals_1 = .5
    y = [1 if random.random() > probability_y_equals_1 else -1 for _ in range(n)]
    x = []

    #creates list of x iid sample vectors based on problem conditions
    for i in range(n):
        x_i = np.zeros(d)
        for j in range(d):
            #first element based on a
            if j == 0:
                if y[i] == 1:
                    x_i[j] = random.uniform(-1,-a)
                else:
                    x_i[j] = random.uniform(a,1)
            #other elements based on [-1,1]
            else:
                x_i[j] = random.uniform(-1,1)
        x.append(x_i)        

    return x, y

def generate_nls_data(m, d, n):
    #create y data
    probability_y_equals_1 = .5
    y = [1 if random.random() >= probability_y_equals_1 else -1 for _ in range(n)]

    #create non-linearly separable data per problem description
    mean = np.zeros(d)
    mean[0] = m
    cov = np.identity(d)
    
    x_arr = np.random.multivariate_normal(mean, cov, n)
    x = [x_arr[i] for i in range(n)]

    return x, y


def perceptron_algorithm(x, y, learning_rate, max_epochs):
    #dimension requirements
    d = int(x[0].size)
    n = len(y)

    convergence_ind = 0
    steps_to_converge = max_epochs
    #random values between 0 and 1 (uniform)
    w = np.random.rand(d)
    b = 0.0

    #function vars
    update_log = np.zeros(max_epochs)
    num_epochs_taken = 0
    num_updates_per_epoch = 0

    while num_epochs_taken < max_epochs:
        for i in range(n):
            y_pred = np.sign(np.dot(w,x[i])+b)
            if y_pred != y[i]:
                w = np.add(w, learning_rate*(y[i]-y_pred)*x[i])
                b = b + learning_rate*(y[i]-y_pred)
                num_updates_per_epoch += 1
        #update vars for next epoch
        update_log[num_epochs_taken] = num_updates_per_epoch
        num_epochs_taken += 1
        num_updates_per_epoch = 0

        #check for convergence
        if update_log[num_epochs_taken-1] == 0:
            steps_to_converge = num_epochs_taken
            num_epochs_taken = max_epochs
            convergence_ind = 1

    return w, b, update_log, steps_to_converge, convergence_ind

def perceptron_algorithm_accuracy(x_test, y_test, w, b):
    n = len(y_test)
    error_count = 0
    for i in range(n):
        y_pred = np.sign(np.dot(w,x_test[i])+b)
        if y_test[i] != y_pred:
            error_count += 1

    error_rate = error_count/n

    return error_count, error_rate



In [16]:
#keep track of linearly separable data
df_log_ls = pd.DataFrame()

In [ ]:
#Linearly separable case

a_values = [.0001, .001, .01, .1, .2]
d_values = [2, 100, 250, 500, 1000]
n_values = [20, 100, 1000, 5000, 7500]

learning_rate = .1
max_epochs = 200

num_of_simulations = 20

#create a log of testing different values
for iter_ in range(num_of_simulations):
    for a in a_values:
        for d in d_values:
            for n in n_values:
                x, y = generate_ls_data(a, d, n)
                w, b, update_log, steps_to_converge, convergence_ind = perceptron_algorithm(x, y, learning_rate, max_epochs)
                x_test, y_test = generate_ls_data(a, d, n)
                error_count, error_rate = perceptron_algorithm_accuracy(x_test, y_test, w, b)
                df_ = pd.DataFrame([[iter_, a, d, n, steps_to_converge, convergence_ind, error_rate]], columns = ['simulation_run','a_value','d_value','n_value','steps_to_converge','convergence_ind','error_rate'])
                df_log_ls = pd.concat([df_log_ls,df_], ignore_index = False)
                df_log_ls.reset_index(drop = True, inplace = True)
                df_log_ls.to_excel(path_out_+'//'+'df_log_linearly_separable.xlsx',index = False)


In [ ]:
#create log of non-linearly separable data
df_log_nls = pd.DataFrame()

In [ ]:
#Non-linearly separable case

m_values = [0, 2, 4]
d_values = [2, 100, 250, 500, 1000]
n_values = [20, 100, 1000, 5000, 7500]

learning_rate = .1
max_epochs = 100

num_of_simulations = 20

#create a log of testing different values
for iter_ in range(num_of_simulations):
    for m in m_values:
        for d in d_values:
            for n in n_values:
                x, y = generate_nls_data(m, d, n)
                w, b, update_log, steps_to_converge, convergence_ind = perceptron_algorithm(x, y, learning_rate, max_epochs)
                error_count, error_rate = perceptron_algorithm_accuracy(x, y, w, b)
                df_ = pd.DataFrame([[iter_, m, d, n, steps_to_converge, convergence_ind, error_rate]], columns = ['simulation_run','m_value','d_value','n_value','steps_to_converge','convergence_ind','error_rate'])
                df_log_nls = pd.concat([df_log_nls,df_], ignore_index = False)
                df_log_nls.reset_index(drop = True, inplace = True)
                df_log_nls.to_excel(path_out_+'//'+'df_log_nonlinearly_separable.xlsx',index = False)


In [18]:
#Non-linearly separable case

m = 2 #or another value greater than 0
d = 1000
n = 7500

x, y = generate_nls_data(m, d, n)
# print("x",x)
# print("y",y)

learning_rate = .1
max_epochs = 100

w, b, update_log, steps_to_converge, convergence_ind = perceptron_algorithm(x, y, learning_rate, max_epochs)

print("convergence indicator",convergence_ind)
print("steps to converge",steps_to_converge)

# print("w",w)
# print("b",b)

error_count, error_rate = perceptron_algorithm_accuracy(x, y, w, b)

print("error rate",error_rate)

convergence indicator 0
steps to converge 100
error rate 0.41933333333333334
